<a href="https://colab.research.google.com/github/allaan08/Deep-Learning-Assignment-2/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# Modify ResNet18 for CIFAR-10
class ResNetCIFAR(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNetCIFAR, self).__init__()
        self.model = resnet18(pretrained=False)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        self.model.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
model = ResNetCIFAR(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training function
def train(epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print(f"Epoch {epoch}, Batch {batch_idx+1}, Loss: {running_loss/100}")
            running_loss = 0.0

# Testing function with metrics
def test(final_epoch=False):
    model.eval()
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100. * correct / total
    print(f"Test Accuracy: {accuracy}%")

    # Only display additional metrics at the final epoch
    if final_epoch:
        conf_matrix = confusion_matrix(all_targets, all_predictions)
        class_report = classification_report(all_targets, all_predictions, output_dict=True)
        precision = class_report["weighted avg"]["precision"]
        recall = class_report["weighted avg"]["recall"]
        f1 = class_report["weighted avg"]["f1-score"]

        # Convert to binary labels for AU-ROC computation for each class
        try:
            roc_auc = roc_auc_score(all_targets, np.array(all_predictions).reshape(-1, 1), multi_class="ovr")
        except ValueError:
            roc_auc = "N/A (single-class prediction issue)"

        print("\nConfusion Matrix:\n", conf_matrix)
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print(f"AU-ROC: {roc_auc}")

# Train and evaluate
for epoch in range(1, 51):  # Run for 50 epochs
    train(epoch)
    if epoch == 50:
        test(final_epoch=True)  # Final epoch with additional metrics
    else:
        test()
    scheduler.step()


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Batch 100, Loss: 2.4208103263378145
Epoch 1, Batch 200, Loss: 1.85990252494812
Epoch 1, Batch 300, Loss: 1.6838390219211579
Test Accuracy: 38.41%
Epoch 2, Batch 100, Loss: 1.4759346628189087
Epoch 2, Batch 200, Loss: 1.3979677259922028
Epoch 2, Batch 300, Loss: 1.3245687448978425
Test Accuracy: 51.8%
Epoch 3, Batch 100, Loss: 1.1447507011890412
Epoch 3, Batch 200, Loss: 1.07042323410511
Epoch 3, Batch 300, Loss: 1.0384661781787872
Test Accuracy: 61.85%
Epoch 4, Batch 100, Loss: 0.904774232506752
Epoch 4, Batch 200, Loss: 0.8900390112400055
Epoch 4, Batch 300, Loss: 0.8785315251350403
Test Accuracy: 66.83%
Epoch 5, Batch 100, Loss: 0.7657712364196777
Epoch 5, Batch 200, Loss: 0.7464985036849976
Epoch 5, Batch 300, Loss: 0.7228716433048248
Test Accuracy: 70.61%
Epoch 6, Batch 100, Loss: 0.6691200521588325
Epoch 6, Batch 200, Loss: 0.6354170340299606
Epoch 6, Batch 300, Loss: 0.6159750410914421
Test Accur

SyntaxError: invalid decimal literal (<ipython-input-1-2a16d7e3e942>, line 1)